<a href="https://colab.research.google.com/github/procoder-cyper/Finetuning_WITH_LORA/blob/main/Finetuning_LORA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch datasets transformers peft bitsandbytes
!pip install -U datasets fsspec



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
import torch

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)

from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
model_name = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,

    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map='auto',
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)

In [ ]:
lora_config = LoraConfig(
    r = 8,
    lora_alpha = 16,
    target_modules = ['q_proj', 'v_proj'],
    lora_dropout = 0.05,
    bias = 'none',
    task_type = TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

In [ ]:
from datasets import load_dataset

# Correct usage
data = load_dataset("openai/gsm8k", "main", split="train[:200]")


In [ ]:
def tokenize(batch):
    texts = [
        f"### Instruction:\n{inst}\n### Response:\n{out}"
        for inst, out in zip(batch['question'], batch['answer'])
    ]

    tokens = tokenizer(
        texts,
        padding = 'max_length',
        truncation = True,
        max_length = 256,
        return_tensors = 'pt'
    )

    tokens['labels'] = tokens['input_ids'].clone()

    return tokens

In [ ]:
tokenize_data = data.map(tokenize, batched=True, remove_columns=data.column_names)
#

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir = './tinyllama-lora-tuned',
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,
    learning_rate = 1e-3,
    num_train_epochs = 50,
    fp16 = True,
    logging_steps = 20,
    save_strategy = 'epoch',
    report_to = 'none',
    remove_unused_columns = False,
    label_names = ["labels"]
)

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenize_data,
    processing_class = tokenizer
)

In [ ]:
trainer.train()

Step,Training Loss
20,1.947500
40,0.909800
60,0.888000
80,1.018100
100,1.226000
120,1.572900
140,2.001000
160,2.539000
180,3.432200
200,3.410100


TrainOutput(global_step=650, training_loss=5.132245677067683, metrics={'train_runtime': 1068.1953, 'train_samples_per_second': 9.362, 'train_steps_per_second': 0.609, 'total_flos': 1.590741172224e+16, 'train_loss': 5.132245677067683, 'epoch': 50.0})

In [ ]:
model.save_pretrained("./tinyllama-lora-tuned-adapter-math")
tokenizer.save_pretrained("./tinyllama-lora-tuned-adapter-math")

('./tinyllama-lora-tuned-adapter-math/tokenizer_config.json',
 './tinyllama-lora-tuned-adapter-math/special_tokens_map.json',
 './tinyllama-lora-tuned-adapter-math/chat_template.jinja',
 './tinyllama-lora-tuned-adapter-math/tokenizer.model',
 './tinyllama-lora-tuned-adapter-math/added_tokens.json',
 './tinyllama-lora-tuned-adapter-math/tokenizer.json')

In [ ]:
import re
from datasets import load_dataset
from tqdm import tqdm

# Load test subset
test_data = load_dataset("openai/gsm8k", "main", split="test[:50]")

def extract_number(text):
    matches = re.findall(r"\d+", text)
    return matches[-1] if matches else ""

generation_args = {
    "max_new_tokens": 128,
    "do_sample": False,
    "temperature": 0.0
}

correct = 0
total = 0

for item in tqdm(test_data):
    prompt = item["question"]
    expected = extract_number(item["answer"])

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, **generation_args)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predicted = extract_number(response)

    if predicted == expected:
        correct += 1
    else:
        print(f"\n❌ Question: {prompt}")
        print(f"🤖 Model Answer: {response}")
        print(f"📤 Extracted: {predicted}")
        print(f"✅ Expected: {expected}")

    total += 1

accuracy = correct / total
print("\n\n=======================")
print(f"✅ Exact Match Accuracy: {accuracy * 100:.2f}%")
print("=======================\n\n")


# New Section